In [ ]:
# @title Start Server (Install)
from IPython.display import clear_output
!pip install pypdf langchain langchain-community langchain-chroma langchain-ollama ollama chromadb PyPDF2 jupyterlab  django
!curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz
!tar -C /usr -xzf ollama-linux-amd64.tgz
from subprocess import Popen
Popen("ollama serve".split())
clear_output()

In [ ]:
# @title Start Server (Download Model)
!ollama pull llama3.2
!ollama pull nomic-embed-text
clear_output()

In [ ]:
# @title Start Server (Initialize)
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
import time

def pdflrn(file,path,embedmodel):
  loader = PyPDFLoader(file)
  data = loader.load()


  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
  all_splits = text_splitter.split_documents(data)


  vectorstore = Chroma.from_documents(all_splits, embedding=OllamaEmbeddings(model=embedmodel), persist_directory=path)
  return True

def pdfres(query,path,model,embedmodel):
  vectorstore = Chroma(embedding_function=OllamaEmbeddings(model=embedmodel), persist_directory=path)
  llm = OllamaLLM(base_url="http://localhost:11434", model=model, verbose=True,
              callbacks=[StreamingStdOutCallbackHandler()])
  retriever = vectorstore.as_retriever()

  template = """ Answer the question based only on the following
  context: {context}

  User: {question}
  Chatbot:
  """
  prompt = PromptTemplate(input_variables=["context","question"], template=template,)
  #memory = ConversationBufferMemory(memory_key="history", return_messages=True, input_key="question")

  qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, verbose=False,
                                        chain_type_kwargs={
                                            #"verbose": True,
                                            "prompt": prompt,
                                            #"memory": memory,
                                        })
  res = qa_chain.invoke({"query": query})
  return res["result"]

In [ ]:
# @title Learn
from google.colab import files
uploaded = files.upload()
for fn in uploaded:
  pdflrn(fn,"data","nomic-embed-text")
clear_output()

In [ ]:
# @title Request a query
query = "" # @param {"type":"string"}
re1 = pdfres(query,"data","llama3.2","nomic-embed-text")

In [ ]:
# @title Django LLM
!git clone https://github.com/epg900/ollama.git
%cd ollama/lamachat/
!ssh-keygen -t rsa -f ~/.ssh/id_rsa <<< y
clear_output()
from threading import Timer
def tt():
    !ssh srv.us -R 1:localhost:8000 -o StrictHostKeyChecking=no
t = Timer(5,tt)
t.start()
!python manage.py runserver